# Langchain: The basics



In [ ]:
!pip -q install openai langchain huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


In [ ]:
import os
from getpass import getpass

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass()


··········


## Plain Conditional Generation

### First with OpenAI GPT3

In [ ]:
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model_name='text-davinci-003',
             temperature=0.9,
             max_tokens = 256)

In [ ]:
text = "Why did the duck cross the road?"

print(llm(text))



To get to the other side.


## Prompt Templates

In [ ]:
from langchain import PromptTemplate

In [ ]:
restaurant_template = """
What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: {review_text}
Sentiment:
"""

review_prompt = PromptTemplate(
    input_variables=["review_text"],
    template=restaurant_template,
)

In [ ]:
reviews = [
    "The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.",
    "Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.",
    "The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!",
    "The food tasted alright, but the tables were not very clean, which was off-putting.",
    "Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit."
]

In [ ]:
print(review_prompt.format(review_text=reviews[0]))


What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment: 



In [ ]:
## querying the model with the prompt template
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=review_prompt)

In [ ]:
chain.run({"review_text": reviews[0]})

'Positive'

In [ ]:
for review in reviews:
    print(prompt.format(review_text=review))
    print(chain.run({"review_text": review}))
    print("================================")


What is the sentiment of the customer review given below?

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment: 

Positive

What is the sentiment of the customer review given below?

review: Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
Sentiment: 

Positive

What is the sentiment of the customer review given below?

review: The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!
Sentiment: 

Positive

What is the sentiment of the customer review given below?

review: The food tasted alright, but the tables were not very clean, which was off-putting.
Sentiment: 

Negative

What is the sentiment of the customer review given below?

review: Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit.
Sentiment: 

Negative


## with Few Shot Learning

In [ ]:
from langchain import FewShotPromptTemplate

In [ ]:
# First, create the list of few shot examples.
examples = [
    {
        "Review": "The grilled chicken was seasoned to perfection and simply melted in the mouth.",
        "Category": "Food Quality"
    },
    {
        "Review": "Despite the crowd, the place was immaculately clean and the restrooms were spotless.",
        "Category": "Overall Hygiene"
    },
    {
        "Review": "The dim lighting and soothing jazz music provided an intimate and romantic setting.",
        "Category": "Restaurant Ambience"
    },
    {
        "Review": "We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.",
        "Category": "Customer Service"
    }
]

In [ ]:
# Prepare the prompt template
example_formatter_template = """
review: {Review}
category: {Category}\n
"""
category_prompt = PromptTemplate(
    input_variables=["Review", "Category"],
    template=example_formatter_template,
)

In [ ]:
print(category_prompt.format(**examples[0]))


review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality




In [ ]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_category_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=category_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the category of review: ",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Review: {review_text}\nCategory:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["review_text"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [ ]:
# We can now generate a prompt using the `format` method.
print(few_shot_category_prompt.format(review_text=reviews[0]))

Give the category of review: 

review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality



review: Despite the crowd, the place was immaculately clean and the restrooms were spotless.
category: Overall Hygiene



review: The dim lighting and soothing jazz music provided an intimate and romantic setting.
category: Restaurant Ambience



review: We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.
category: Customer Service


Review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Category:


In [ ]:
from langchain.chains import LLMChain

category_chain = LLMChain(llm=llm, prompt=few_shot_category_prompt)

print(reviews[0])
# Run the chain only specifying the input variable.
print(category_chain.run(reviews[0]))

The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
 Food Quality & Restaurant Ambience


In [ ]:
for review in reviews:
    print(review)
    print(category_chain.run({"review_text": review}))
    print("================================")

The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
 Food Quality and Restaurant Ambience
Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
 Customer Service
The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!
 Restaurant Decor
The food tasted alright, but the tables were not very clean, which was off-putting.
 Cleanliness
Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit.
 Overall Experience


In [ ]:
from langchain.schema import StrOutputParser

In [ ]:
review_chain = review_prompt | llm | StrOutputParser()

In [ ]:
review_chain.invoke({"review_text": reviews[0]})

'Positive'

In [ ]:
category_chain = few_shot_category_prompt | llm | StrOutputParser()

In [ ]:
category_chain.invoke({"review_text": reviews[1]})

' Customer Service'

In [ ]:
response_to_customer = """
"You received a {feedback} feedback from a customer about {category}.
Write an appropriate response to the customer."""

In [ ]:
response_prompt = PromptTemplate(
    input_variables=["feedback", "category"],
    template=response_to_customer,
)

In [ ]:
final_response_chain = response_prompt | llm | StrOutputParser()

In [ ]:
complete_chain = (
    {
        "feedback": review_chain,
        "category": category_chain,
    }
    | final_response_chain
)

In [ ]:
from pprint import pprint

In [ ]:
pprint(complete_chain.invoke({"review_text": reviews[4]}))

('\n'
 '\n'
 'I apologize that your overall experience was not up to your expectations. We '
 'take customer satisfaction very seriously, and I want to assure you that we '
 'will investigate what happened and take steps to ensure this type of issue '
 'does not occur again. If there is anything we can do to make it up to you, '
 'please let us know. Thank you for bringing this issue to our attention.')
